# 0. Setup

In [1]:
%  .txt file chooser (1, 2, 3 or 4)
file_n = 1;

switch file_n
    case 1
        filename="ed-p05/links_car_manufacturer.txt";
    case 2
        filename="ed-p05/links_java_coffee.txt";
    case 3
        filename="ed-p05/links_search_engines.txt";
    case 4
        filename="ed-p05/links_stock_market.txt";
    otherwise 
        error('I only understand numbers 1, 2, 3 and 4!');
end

%  how many iterations of HITS algo do you want me to do?
iterations = 3;

# 1. Reading .txt fie dimentions

In [2]:
%  reading file
file_id = fopen(filename, 'r');

line = fgetl(file_id);

%  getting dimentions of a future cell array
nrows = 0;
max_links = size(strsplit(line, " "), 2);
while ischar(line)
    nrows = nrows + 1;
    
    curr_links = size(strsplit(line, " "), 2);
    
    %  easiest max value finder imaginable
    if curr_links > max_links
        max_links = curr_links;
    end
    
    line = fgetl(file_id);
end
ncols = max_links;

%  closing file
fclose(file_id);

[nrows, ncols]


ans =

   858    84



# 2. Creating link dataset (cell array)

In [3]:
%  reading file
file_id = fopen(filename, 'r');

line = fgetl(file_id);

%  creating cell array to save modified links in some structure
file_array = cell(nrows, ncols);
file_array(:) = {'NaN'};

%  creating cell array to save og links to print at the end
og_files = cell(nrows, ncols);
og_files(:) = {'NaN'};

i = 0;

%  iterating through every line
while ischar(line)
    
    i = i+1;
    
    %  prefixes to remove
    str2remove = ["http://", "https://", "www."];
    
    %  links from the whole line
    links = strsplit(line, " ");
    
    %  iterating through every link in a line
    for k=1:size(links, 2)
        
        %  particular link
        link = links{k};
        
        %  adding og link to a cell array
        og_files{i, k} = link; 

        %  iterating through every prefix to remove
        for j=1:size(str2remove, 2)
        
            wrong_str = str2remove(:, j);

            %  removing prefix
            if startsWith(link, wrong_str)
                link = char(strrep(link, wrong_str, ''));
            end
            
            %  making sure every link is neihter string array nor cell array
            if isstring(link) || iscell(link)
                link = char(link);
            end
        
        end
        
        %  removing sufixes
        if endsWith(link, "index.html")
            link = char(strrep(link, "index.html", ''));
        elseif endsWith(link, "index.htm")
            ink = char(strrep(link, "index.htm", ''));
        end
        
        if endsWith(link, "/")
            link = char(strrep(link, "/", ''));
        end
        
        %  inserting link to a cell array
        file_array{i, k} = link;
    end
    %  getting another line
    line = fgetl(file_id);
end

%  closing file
fclose(file_id);

%  printing head of a created array
file_array(1:8 , 1:3)

%  prining its size
size(file_array)


ans =

  8x3 cell array

    {'car-stuff.com/c...'}    {'NaN'               }    {'NaN'               }
    {'ford.co.uk'        }    {'NaN'               }    {'NaN'               }
    {'saab.com'          }    {'NaN'               }    {'NaN'               }
    {'consumeraction....'}    {'pueblo.gsa.gov/...'}    {'ConsumerAction.gov'}
    {'pueblo.gsa.gov/...'}    {'NaN'               }    {'NaN'               }
    {'ConsumerAction.gov'}    {'NaN'               }    {'NaN'               }
    {'psa-peugeot-cit...'}    {'psa-peugeot-cit...'}    {'psa.fr'            }
    {'psa-peugeot-cit...'}    {'NaN'               }    {'NaN'               }


ans =

   858    84



# 3. Getting unique links

In [4]:
%  getting unique links
unique_links = string(unique(file_array(:)));

%  removing "NaN" added at the end of each row with less than ncols columns
unique_links = unique_links(unique_links ~= "NaN");

%  prining first 8 unique links to show everything works (hopefully)
unique_links(1:8)

%  printing how many unique link there is 
unique_links_number = size(unique_links, 1)


ans = 

  8x1 string array

    "1-auto-car-loans.com/jaguar-customer-relations.htm"
    "1-debt-consolidation-debt-reduction-service.com/link-dir/investing.html"
    "100autoguias.com/es/automotriz/marcas/honda/n.htm"
    "100links.supereva.ittempo_liberomotoriautocostruttorieuropei"
    "15d.ru/15d/links.shtml"
    "195.92.249.252/~kgb/Caterham7/mycaterham.html"
    "1hg.com"
    "1stopcaradvice.co.uk/car_manufacturers.asp"


unique_links_number =

   913



# 4. HITS Table - Iteration 0

In [5]:
%  creating interation-0 table for the algo to start with
old_auth = ones(1, unique_links_number);
old_hub = ones(1, unique_links_number);

new_auth = zeros(1, unique_links_number);
new_hub = zeros(1, unique_links_number);

hits = table(unique_links, old_auth', old_hub', new_auth', new_hub');

hits.Properties.VariableNames = {'Link', 'Old_Authority', 'Old_Hub', 'New_Authority', 'New_Hub'};

head(hits)

                                      Link                                       Old_Authority    Old_Hub    New_Authority    New_Hub
    _________________________________________________________________________    _____________    _______    _____________    _______

    "1-auto-car-loans.com/jaguar-customer-relations.htm"                               1             1             0             0   
    "1-debt-consolidation-debt-reduction-service.com/link-dir/investing.html"          1             1             0             0   
    "100autoguias.com/es/automotriz/marcas/honda/n.htm"                                1             1             0             0   
    "100links.supereva.ittempo_liberomotoriautocostruttorieuropei"                     1             1             0             0   
    "15d.ru/15d/links.shtml"                                                           1             1             0             0   
    "195.92.249.252/~kgb/Caterham7/mycaterham.html"          

In [6]:
%  1st column of an array -> "main" nodes that point to another nodes
main_sites = file_array(:, 1);

%  2nd-84th column of an array -> nodes that are pointed to from "main" nodes
pointed_sites = file_array(:, 2:end);

# 5. HITS Algorithm

In [7]:
for x=1:iterations

    %  -----------  AUTHORITY -----------
    
    %  progres bars init
    h1 = waitbar(0, 'Loading...');
    
    %  new authority column
    new_auth = [];

    %  iterating through every unique link in network
    for i=1:size(hits, 1)

        %  current link
        link = hits{i, 1};

        %  binary vector:
        %  1 - link is in the row (ergo this row's "main" link points to the current link),
        %  0 - link is NOT in the row (this row's "main" link DOES NOT point to the current link)
        points2link = any(ismember(pointed_sites, link), 2);

        %  "main" links pointing to the current link
        pointing_links = main_sites(points2link);

        %  if no link points to current link, the authority is 0, end the iteration  
        if isempty(pointing_links)
            new_auth = [new_auth, 0];
            continue;
        end

        %  if something points to a current link, calculate its authority
        auth = 0;

        %  iterating through every "main" link pointing to the current link
        for j=1:size(pointing_links, 1)

            %  j-th "main" link
            pointing_website = pointing_links{j};

            %  finding its row number
            row_id = find(strcmp(hits.Link, pointing_website));

            %  collecting the "main" link's old hub score (from last iteration)
            old_hub_score = hits{row_id, 3};

            %  new authority score is the sum of old hub scores of "main" links pointing to a current link
            auth = auth + old_hub_score;

        end

        %  filling the new column
        new_auth = [new_auth, auth];
        
        %  progres bar update
        waitbar(i/size(hits, 1), h1, sprintf('Calculating authority scores - Iteration %d ... %d%%', x, round(i/size(hits, 1)*100)));
        
    end

    %  normalizing values 
    new_auth_norm = new_auth ./ sum(new_auth);

    %  updating column
    hits.New_Authority = new_auth_norm';
    
    %  closing progres bar
    close(h1);
    
    %  ----------  HUB  ----------
    
    % progres bar init
    h2 = waitbar(0, 'Loading...');
    
    % new hub collumn
    new_hubs = [];

    %  iterating through every unique link
    for i=1:size(hits,1)

        %  current link
        link = hits{i, 1};

        %  array of the links current link points to
        pointed_by_link = pointed_sites(main_sites==link, :);

        %  if current link DOES NOT point to any other link, the hub score is 0, end the iteration
        if isempty(pointed_by_link)
            new_hubs = [new_hubs, 0];
            continue;
        end

        %  if current link points to any other link, calculate its new hub score
        hub = 0;

        %  iterating through every link current link points to
        for j=1:size(pointed_by_link, 2)

            %  j-th link our current link points to
            pointed_website = pointed_by_link{j};

            %  if j-th pointed link is "NaN" (so every next link is also "NaN"), end calculating hub scorew 
            if pointed_website=="NaN"
                break;
            end

            %  find pointed link's row id
            row_id = find(strcmp(hits.Link, pointed_website));

            %  obtain its old auth score (from the previous iteration)
            old_auth_score = hits{row_id, 2};

            %  new hub score is the sum of all the old authority scores of the pointed links
            hub = hub + old_auth_score;

        end

        %  filling new hub collumn
        new_hubs = [new_hubs, hub];
        
        %  progres bar update
        waitbar(i/size(hits, 1), h2, sprintf('Calculating hub scores - Iteration %d ... %d%%', x, round(i/size(hits, 1)*100)));
        
    end

    %  normalizing new hub scores
    new_hubs_norm = new_hubs ./ sum(new_hubs);

    %  updating the column
    hits.New_Hub = new_hubs_norm';
    
    %  closing progres bar
    close(h2);
    
    %  swaping HITS table columns for the next iteration
    hits.Old_Authority = hits.New_Authority;
    hits.Old_Hub = hits.New_Hub;
    
end

%  formating table to obtain relevant authority data
best_authority = removevars(hits, {'Old_Authority', 'Old_Hub', 'New_Hub'});
best_authority = sortrows(best_authority, 'New_Authority', 'descend');
best_authority.Properties.VariableNames = {'Link', 'Authority Score'};

%  formating table to obtain relevant hub data
best_hub = removevars(hits, {'Old_Authority', 'Old_Hub', 'New_Authority'});
best_hub = sortrows(best_hub, 'New_Hub', 'descend');
best_hub.Properties.VariableNames = {'Link', 'Hub Score'};

%  printing the results
fprintf("\n----- HITS algorithm results -----");

txt_name = strsplit(filename, "/");
txt_name = txt_name{2}; 
fprintf("\n\nFilename:  %s \n", txt_name);

fprintf("\nNumber of iterations: %d \n\n", iterations)

disp("↓↓ Sites with best authority ↓↓");
head(best_authority, 10);

disp("↓↓ Sites with best hub ↓↓");
head(best_hub, 10);


----- HITS algorithm results -----

Filename:  links_car_manufacturer.txt 

Number of iterations: 3 

↓↓ Sites with best authority ↓↓
         Link          Authority Score
    _______________    _______________

    "ford.com"            0.024539    
    "honda.com"           0.022389    
    "cadillac.com"        0.021668    
    "chevrolet.com"       0.021493    
    "pontiac.com"         0.020939    
    "buick.com"           0.020776    
    "toyota.com"          0.020451    
    "gm.com"               0.02018    
    "opel.com"            0.019598    
    "vw.com"              0.019117    

↓↓ Sites with best hub ↓↓
                                                Link                                                Hub Score
    ____________________________________________________________________________________________    _________

    "grocerygetters.com/resource-auto-makers.shtml"                                                 0.020734 
    "world-auto.de/links/Herstellerlin

*Karolian Węglowska, Krzysztof Stawarz*  
*Kraków, 22.05.2023*